In [108]:
# Audio File Directory Locations and Names
input_file_non_wav = "./audio/two.m4a"
non_wav_format = "m4a"
input_file = "./audio/two.wav"
output_file = "./audio/two_20kHz.wav"

In [109]:
# If needed to convert into WAV
from pydub import AudioSegment
audio = AudioSegment.from_file(input_file_non_wav, format=non_wav_format)
audio.export(input_file, format="wav")

<_io.BufferedRandom name='./audio/two.wav'>

In [110]:
# Imports
import numpy as np
from scipy.io.wavfile import read, write
from scipy.signal import butter, filtfilt

In [111]:
# Load Audio FIle
fs, audio_signal = read(input_file)

In [112]:
# Make Sure Audio Is Mono
if len(audio_signal.shape) > 1:
    audio_signal = audio_signal[:, 0]

In [113]:
# Normalize Audio
audio_signal = audio_signal / np.max(np.abs(audio_signal))

In [114]:
# Generate High-Frequency Carrier
duration = len(audio_signal) / fs
t = np.linspace(0, duration, len(audio_signal), endpoint=False)
carrier_frequency = 20000
carrier_wave = np.sin(2 * np.pi * carrier_frequency * t)

In [115]:
# Pass Filter (Dual Purpose: LPF / HPF)
def pass_filter(type, signal, cutoff, fs, order=8):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype=type, analog=False)
    return filtfilt(b, a, signal)

In [116]:
# Apply LPF to limit audio signal to 3 kHz (Acceptable Audible Hearing Range)
audio_signal = pass_filter('low', audio_signal, cutoff=3000, fs=fs)
audio_signal = audio_signal / np.max(np.abs(audio_signal))

In [117]:
# Modulate
modulated_signal = (1 + audio_signal) * carrier_wave

# Apply HPF to clear audio still roughly remaining in hearing range (Beneath 17000)
modulated_signal = pass_filter('high', modulated_signal, cutoff=17000, fs=fs)

# Normalize Audio
modulated_signal = modulated_signal / np.max(np.abs(modulated_signal))

In [118]:
# Write Output File
write(output_file, fs, (modulated_signal * 32767).astype(np.int16))